     Transformer architecture has encoder and decoder stack, hence called encoder-decoder architecture whereas BERT is just an encoder stack of transformer architecture. There are two variants, BERT-base and BERT-large, which differ in architecture complexity. The base model has 12 layers in the encoder whereas the Large has 24 layers.

       BERT was trained on a large text corpus, which gives architecture/model the ability to better understand the language and to learn variability in data patterns and generalizes well on several NLP tasks. As it is bidirectional that means BERT learns information from both the left and the right side of a token’s context during the training phase

In [2]:
!pip install -q transformers

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import BertTokenizer

### Load IMDB Data 

In [5]:
(ds_train, ds_test), ds_info = tfds.load('imdb_reviews', split = (tfds.Split.TRAIN, tfds.Split.TEST),
                                          as_supervised=True, with_info=True)

In [6]:
ds_info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset.
    This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='C:\\Users\\Shivani Dussa\\tensorflow_datasets\\imdb_reviews\\plain_text\\1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=2500

In [7]:
type(ds_train),type(ds_test)

(tensorflow.python.data.ops.dataset_ops.PrefetchDataset,
 tensorflow.python.data.ops.dataset_ops.PrefetchDataset)

In [8]:
ds_train

<PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>

In [9]:
len(ds_train), len(ds_test)

(25000, 25000)

### Explore IMDB Data

In [10]:
type(ds_train)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [11]:
tfds.as_numpy(ds_train.take(5))

In [12]:
for data in tfds.as_numpy(ds_train.take(5)):
    print(data)    

(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", 0)
(b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot developme

##### Observation:
          - Here we taken first 5 reviews from imdb reviews data.. after the sentence it also given whether the sentence is positive(1) or negitive(0)

In [13]:
# here, we are dividing that review sentence and label whether the sentence is +ve or -ve
for review, label in tfds.as_numpy(ds_train.take(1)):
    print(review)
    print("------------------------------------------------------------------------------------------------------------------")
    print("type:",type(review))
    print("------------------------------------------------------------------------------------------------------------------")
    print(label)
    print("type:",type(label))  

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
------------------------------------------------------------------------------------------------------------------
type: <class 'bytes'>
------------------------------------------------------------------------------------------------------------------
0
type: <class 'numpy.int64'>


In [20]:
reviews = []
labels= []
for review, label in tfds.as_numpy(ds_train.take(6)):    # taking 6 reviews i.e., 0-5
    reviews.append(review.decode())
    labels.append(label)
    print(review.decode()[0:56], '\t', label)     # this is 0:50 means it takes sentences of words space also means to a 1 word

This was an absolutely terrible movie. Don't be lured in 	 0
I have been known to fall asleep during films, but this  	 0
Mann photographs the Alberta Rocky Mountains in a superb 	 0
This is the kind of film for a snowy Sunday afternoon wh 	 1
As others have mentioned, all the women that go nude in  	 1
This is a film which should be seen by anybody intereste 	 1


**Here, 0,1,2 indexes sentences are negitive and 3,4 indexes are positive**

In [21]:
reviews[2]

'Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Mountie telling the people of Dawson City, Yukon to elect themselves a marshal (yes a marshal!) and to enforce the law themselves, then gunfighters battling it out on the streets for control of the town? <br /><br />Nothing even remotely resembling that happened on the Canadian side of the border during the Klondike gold rush. Mr. Mann and company appear to have mistaken Dawson City for Deadwood, the Canadian North for the American Wild West.<br /><br />Canadian viewers be prepared for a Reefer Madness type of enjoyable howl with this ludicrous plot, or, to shake your head in disgust.'

In [23]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis",'distilbert-base-uncased-finetuned-sst-2-english')

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [32]:
print(classifier(reviews[0]))   # predicts correctly 0.99
print(classifier(reviews[1]))   # predicts correctly 0.99
print(classifier(reviews[2]))   # predicts wrongly 0.66 its a negitive sentence but its shows positive with 0.66 accuracy score
print(classifier(reviews[3]))   # predicts correctly 0.99
print(classifier(reviews[4]))   # predicts correctly 0.99
print(classifier(reviews[5]))   # predicts correctly 0.99

[{'label': 'NEGATIVE', 'score': 0.999795138835907}]
[{'label': 'NEGATIVE', 'score': 0.9992781281471252}]
[{'label': 'POSITIVE', 'score': 0.664378821849823}]
[{'label': 'POSITIVE', 'score': 0.9997604489326477}]
[{'label': 'POSITIVE', 'score': 0.999556839466095}]
[{'label': 'POSITIVE', 'score': 0.9994988441467285}]


### Tokenizer
Let’s prepare the data according to the format needed for the BERT model

In [36]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case = True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

C:\Anaconda\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Shivani Dussa\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [47]:
# can be upto 512  for berts
max_length = 512
batch_size = 6

In [48]:
def convert_example_to_feature(review):
    return tokenizer.encode_plus(review,add_special_tokens= True,    # add [CLS] [SEP]
                                max_length = max_length,             #max length of the text that can go to BERT
                                pad_to_max_length = True,            #add [PAD] tokens
                                return_attention_mask= True,         #add atention mask to not focus on pad tokens
                                )

In [49]:
for review, label in tfds.as_numpy(ds_train.take(1)):
    encodedReview = convert_example_to_feature(review.decode())
    print("type:",type(encodedReview))
    print("----------------------------------------------------------------------------------------------------------------")
    print("keys:",encodedReview.keys())
    print("----------------------------------------------------------------------------------------------------------------")
    #print(encodedReview)
    print(encodedReview['input_ids'])
    print("----------------------------------------------------------------------------------------------------------------")
    print(encodedReview['token_type_ids'])
    print("----------------------------------------------------------------------------------------------------------------")
    print(encodedReview['attention_mask'])
    print("----------------------------------------------------------------------------------------------------------------")
    print(review.decode())
    print("----------------------------------------------------------------------------------------------------------------")
    print(label)

type: <class 'transformers.tokenization_utils_base.BatchEncoding'>
----------------------------------------------------------------------------------------------------------------
keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
----------------------------------------------------------------------------------------------------------------
[101, 2023, 2001, 2019, 7078, 6659, 3185, 1012, 2123, 1005, 1056, 2022, 26673, 1999, 2011, 5696, 3328, 2368, 2030, 2745, 3707, 7363, 1012, 2119, 2024, 2307, 5889, 1010, 2021, 2023, 2442, 3432, 2022, 2037, 5409, 2535, 1999, 2381, 1012, 2130, 2037, 2307, 3772, 2071, 2025, 2417, 21564, 2023, 3185, 1005, 1055, 9951, 9994, 1012, 2023, 3185, 2003, 2019, 2220, 3157, 7368, 2149, 10398, 3538, 1012, 1996, 2087, 17203, 5019, 2020, 2216, 2043, 1996, 25882, 8431, 2020, 2437, 2037, 3572, 2005, 25239, 1012, 3814, 9530, 5428, 2696, 17649, 2596, 6887, 16585, 1010, 1998, 2014, 18404, 1011, 2293, 6771, 2007, 3328, 2368, 2001, 2498, 2021, 1037, 17203, 

##### Input IDs:   
          The input ids are often the only required parameters to be passed to the model as input. Token indices, numerical representations of tokens building the sequences that will be used as input by the model.

##### Attention mask:
          Attention Mask is used to avoid performing attention on padding token indices. Mask value can be either 0 or 1, 1 for tokens that are NOT MASKED, 0 for MASKED tokens.

##### Token type ids: 
         It is used in use cases like sequence classification or question answering. As these require two different sequences to be encoded in the same input IDs. Special tokens, such as the classifier[CLS] and separator[SEP] tokens are used to separate the sequences.

### Encode Data 

In [51]:
# this is helper function to transform our raw data to an appropriate format ready to feed into the BERT model
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
    return {
        "input_ids":input_ids,
        "token_type_ids":token_type_ids,
        "attention_mask":attention_masks,
    }, label

def encode_examples(ds, limit = -1):
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    if (limit >0):
        print("Using", limit, "records from ds")
        ds = ds.take(limit)
    else:
        print("Using all records from ds")
    for review, label in tfds.as_numpy(ds):
        bert_input = tokenizer.encode_plus(review.decode(),add_special_tokens = True,max_length = max_length,
                                           pad_to_max_length = True,return_attention_mask = True,)
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])
    return tf.data.Dataset.from_tensor_slices((input_ids_list,attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)
        

- **batch** - Combines consecutive elements of this dataset into batch
- **shuffle** - Randomly shuffles the elements of this dataset. This dataset fills a buffer with buffer_size elements, then randomly samples elements from this buffer, replacing the selected elements with new elements.

In [52]:
# train dataset
ds_train_encoded = encode_examples(ds_train, limit = 5000).shuffle(2000).batch(batch_size)
# test dataset
ds_test_encoded = encode_examples(ds_test, limit = 1000).batch(batch_size)

Using 5000 records from ds


C:\Anaconda\lib\site-packages\transformers\tokenization_utils_base.py:2302: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Using 1000 records from ds


### Understanding ds_train encoded structure

In [53]:
for record in ds_train_encoded.take(1).as_numpy_iterator():
    print(type(record))

<class 'tuple'>


In [54]:
len(record)

2

In [55]:
record

({'input_ids': array([[  101,  2013,  1996, ...,  1997, 15587,   102],
         [  101,  2092,  1010, ...,     0,     0,     0],
         [  101,  1996,  2034, ...,     0,     0,     0],
         [  101,  7004, 13573, ...,     0,     0,     0],
         [  101,  1045,  2387, ...,     0,     0,     0],
         [  101,  1045,  2074, ...,     0,     0,     0]]),
  'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]),
  'attention_mask': array([[1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0]])},
 array([[1],
        [0],
        [1],
        [0],
        [1],
        [0]]))

In [56]:
# here, we are separating as inputs,token_ids,attention mask as inputs and label as labels 
for inputs, labels in ds_train_encoded.take(1).as_numpy_iterator():
    print(type(inputs))
    print(type(labels))

<class 'dict'>
<class 'numpy.ndarray'>


In [62]:
len(inputs), len(labels)

(3, 6)

In [61]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [67]:
for inputs, labels in ds_train_encoded.take(1).as_numpy_iterator():
    print(inputs.keys())
    print(len(labels))

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
6


In [68]:
for inputs, labels in ds_train_encoded.take(1).as_numpy_iterator():
    print(len(inputs['input_ids']))
    print(len(inputs['token_type_ids']))
    print(len(inputs['attention_mask']))
    print(len(labels))

6
6
6
6


In [73]:
inputs['token_type_ids']

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [74]:
inputs['attention_mask']

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [75]:
inputs['input_ids']

array([[ 101, 2178, 2028, ...,    0,    0,    0],
       [ 101, 1045, 2074, ...,    0,    0,    0],
       [ 101, 1045, 6719, ...,    0,    0,    0],
       [ 101, 1999, 2344, ...,    0,    0,    0],
       [ 101, 1045, 3427, ...,    0,    0,    0],
       [ 101, 2045, 2003, ...,    0,    0,    0]])

**As we have selected batch_size=6, we see each data record consists of 6 encoded reviews & 6 labels**

In [76]:
for inputs, labels in ds_train_encoded.take(1).as_numpy_iterator():
    print(inputs['input_ids'][0])
    print(inputs['token_type_ids'][0])
    print(inputs['attention_mask'][0])
    print(labels[0])

[  101  1996 13130  7065 16116  2105  1037  2611  2040  2442  4553  2000
  2022  3625  2005  2014  2219  4506  1012  2004  2016  2018  1996  2373
  1997  3894  1010  2016  2411  2109  2009  2000  3046  2000  2393  2014
  3866  3924  2030  2841  1010  4703  4525  1999 18204 26136  2015  2008
  2024  2411 16775  1998  2467 14742  1012  1026  7987  1013  1028  1026
  7987  1013  1028  1996  2565  2211  2007 21876  1005  1055  7357  1999
  2152  2082  1999  1996  7214  2237  1997  2225  6374  1010  2284  2379
  3731  1010  4404  1006  2004  4941  2000  2665  5634  1999  1996  5888
  1007  1012  1999  1996  2186  1005  2101  3692  1010 21876  3852  2013
  2152  2082  1998  8302  1999  2267  1010  2059  2333  2006  2000  2014
  4740  2000  2444  2006  2014  2219  1998  2562  1037  3105  2012  1996
  2334  3780  1012  4911  2582  2013  2049  5021  6147  1010  1996  2265
  3092  2007 21876  1005  1055  5030  1010  2348  1010  1999  1996  2203
  1010  2016  4704  1996  5030  1998  2743  2125  2

### Link to Google Device

In [80]:
from google.colab import drive
drive.mount('https://drive.google.com/drive/my-drive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

ModuleNotFoundError: No module named 'google.colab'

### Creating and Training Fine Tuning Model

In [82]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf

learning_rate = 2e-5  #recommended learning rate for Adam 5e-5, 3e-5, 2e-5
number_of_epochs = 1 # taking 1 epoch 

#model initializational
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [83]:
# Choosing Adam Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate, epsilon = 1e-08)
# we do not have one-hot vectors, here we can use sparse categorical crossentrophy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer = optimizer, loss = loss, metrics = [metric])

In [84]:
bert_history = model.fit(ds_train_encoded, epochs = number_of_epochs, validation_data = ds_test_encoded)

 12/834 [..............................] - ETA: 11:09:41 - loss: 0.7013 - accuracy: 0.5417


KeyboardInterrupt



In [123]:
!copy /mydrive/fineTuneBERTwithIMDB_weights.* /content

The syntax of the command is incorrect.


In [85]:
model.save_weights('fineTuneBERTwithIMDB_weights.ckpt')

### Use previously Trained weights(instead of training)

In [113]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
model.load_weights('fineTuneBERTwithIMDB_weights.ckpt')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Inference on random sample

In [114]:
myreview = "Though the movie was good, it was a bit too long"
encoded_myreview = tokenizer.encode(myreview, truncation=True, padding=True, return_tensors="tf")

In [115]:
tf_output = model.predict(encoded_myreview)[0]
tf_prediction = tf.nn.softmax(tf_output,axis = 1)
labels = ['Negative','Positive'] #(0:negative, 1:positive)
label = tf.argmax(tf_prediction, axis = 1)
label = label.numpy()
print(labels[label[0]])

Negative


In [116]:
print(tf_output)
print(tf_prediction)

[[0.16335757 0.14577296]]
tf.Tensor([[0.504396   0.49560395]], shape=(1, 2), dtype=float32)


### Inference on Test Data

In [117]:
reviews = []
labels = []
for review, label in tfds.as_numpy(ds_test.take(5)):
    reviews.append(review.decode())
    labels.append(label)
    print(review.decode()[0:50], '\t', label)

There are films that make careers. For George Rome 	 1
A blackly comic tale of a down-trodden priest, Naz 	 1
Scary Movie 1-4, Epic Movie, Date Movie, Meet the  	 0
Poor Shirley MacLaine tries hard to lend some grav 	 0
As a former Erasmus student I enjoyed this film ve 	 1


In [118]:
i = 0
reviews[i]

"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come."

In [108]:
i = 1
reviews[i]

"A blackly comic tale of a down-trodden priest, Nazarin showcases the economy that Luis Bunuel was able to achieve in being able to tell a deeply humanist fable with a minimum of fuss. As an output from his Mexican era of film making, it was an invaluable talent to possess, with little money and extremely tight schedules. Nazarin, however, surpasses many of Bunuel's previous Mexican films in terms of the acting (Francisco Rabal is excellent), narrative and theme.<br /><br />The theme, interestingly, is something that was explored again in Viridiana, made three years later in Spain. It concerns the individual's struggle for humanity and altruism amongst a society that rejects any notion of virtue. Father Nazarin, however, is portrayed more sympathetically than Sister Viridiana. Whereas the latter seems to choose charity because she wishes to atone for her (perceived) sins, Nazarin's whole existence and reason for being seems to be to help others, whether they (or we) like it or not. The

In [119]:
myreview = reviews[i]
encoded_myreview = tokenizer.encode(myreview, truncation = True, padding = True, return_tensors = "tf")

In [120]:
tf_output = model.predict(encoded_myreview)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1)
labels = ['Negative','Positive'] #(0:negative, 1:positive)
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print(labels[label[0]])

Negative


In [121]:
print(tf_output)
print(tf_prediction)

[[0.097545   0.05821977]]
tf.Tensor([[0.50983006 0.49017   ]], shape=(1, 2), dtype=float32)


In [112]:
classifier(myreview)

[{'label': 'POSITIVE', 'score': 0.9988347887992859}]

tokenizer. encode will encode our test example into integers using Bert tokenizer, then we use predict method on the encoded input to get our predictions. The model. predict will return logits, on which we can apply softmax function to get the probabilities for each class, and then using TensorFlow argmax function we can get the class with the highest probability and map it to text labels (positive or negative).

### Extra code -- to understand how tf.data.Dataset.from_tensor_slices() works

In [103]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
for element in dataset:
    print(element)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)


In [104]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
dataset = dataset.map(lambda x: x*2)
list(dataset.as_numpy_iterator())

[2, 4, 6]

In [105]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
print(list(dataset.as_numpy_iterator()))

[1, 2, 3]


In [106]:
dataset = tf.data.Dataset.from_tensor_slices({'a': ([1, 2], [3, 4]),
                                              'b': [5, 6]})
list(dataset.as_numpy_iterator()) == [{'a': (1, 3), 'b': 5},
                                      {'a': (2, 4), 'b': 6}]

True

In [107]:
list(dataset.as_numpy_iterator())

[{'a': (1, 3), 'b': 5}, {'a': (2, 4), 'b': 6}]